<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

In [1]:
from eoxhub import check_compatibility
check_compatibility("user-2022.10-14", dependencies=[])

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
import urllib.request as request
import h5py
import os
from ipywidgets import widgets, IntSlider, jslink, interact, interactive, fixed, interact_manual
import markdown
from erddapy import ERDDAP
from ipyleaflet import Map, Marker, GeoData, ImageOverlay, basemaps, basemap_to_tiles, LayersControl, ScaleControl, FullScreenControl, WidgetControl
# import pandas as pd
import numpy as np
# from IPython.display import display
from netCDF4 import num2date
# from datetime import datetime
import geopandas as gpd
########################################################
import sys
import pandas as pd
import ipywidgets as widgets
from IPython.display import Markdown, HTML, Javascript, display, Image
import subprocess
import csv
from __future__ import print_function
# from ipywidgets import interact, interactive, fixed, interact_manual
import csv
import re
import warnings
#from init import *
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
def printmd(string):
    display(Markdown(string))
###############################################################
from ftplib import FTP
import multiprocessing as mlp
import shutil
import tempfile
import urllib.request
from datetime import datetime, timedelta

from osgeo import gdal
# import numpy as np
from osgeo import ogr
from osgeo import osr
# import pandas as pd
import pyproj

from multiprocessing import Manager
# from ipywidgets import widgets, IntSlider, jslink
from ipyleaflet import Map, projections, GeoData, basemap_to_tiles, basemaps, WidgetControl, ScaleControl, FullScreenControl, LayersControl #, ImageOverlay, 
import geopandas as gpd
import leafmap
import altair as alt
# import localtileserver  # was needed localy to be able to add raster on the leafmap Map.

ModuleNotFoundError: No module named 'erddapy'

In [ ]:
class merge_btl_nutrient:
    
    def get_btlfile(self, btl_url, wdir):
        request.urlretrieve(btl_url, os.path.join(wdir, "btl.h5"))
        return 0

    def btl_to_dataframe(self, wdir):
        f = h5py.File(os.path.join(wdir, "btl.h5"), 'r')
        df_btl = pd.DataFrame()
        for grp in f:
            tempo_list = []
            tempo_columns = []
            for c in f[grp]:
                tempo_columns.append(c)
                tempo_list.append(f[grp][c])
            list_array = np.transpose(np.array(tempo_list))
            tempo_df = pd.DataFrame(list_array, columns=tempo_columns)
            tempo_df['station'] = [f[grp].attrs['Station'].strip().split(' ')[-1]] * len(tempo_df)
            tempo_df['cast'] = [int(f[grp].attrs['Cast_Number'].strip())] * len(tempo_df)
            df_btl = pd.DataFrame.append(tempo_df, df_btl)
        f.close()
#         # added to extract a csv format of the btl.h5 data to send out to the GUI team
#         df_btl.to_csv(os.path.join(wdir, "btl.csv"), header=1, index=0)
        #######
        return df_btl

    def merge(self, df_nutrient, df_btl, file_ge_btl):
        ge_time_header = [i for i in list(df_nutrient.columns) if
                          ((i.lower().__contains__('date')) or 
                           (i.lower().__contains__('time')))].pop()
        btl_time_header = [i for i in list(df_btl.columns) if
                           ((i.lower().__contains__('date')) or 
                           (i.lower().__contains__('time')))].pop()
        ge_station_header = [i for i in list(df_nutrient.columns) if
                             i.lower().__contains__('station')].pop()
        btl_station_header = [i for i in list(df_btl.columns) if
                              i.lower().__contains__('station')].pop()
        ge_bottle_header = [i for i in list(df_nutrient.columns) if
                            (i.lower().__contains__('bottle'))].pop()
        """ TODO: bopo should be replaced with RosPos when the data with the corrected variable name will be served on Hyrax."""
        btl_bottle_header = [i for i in list(df_btl.columns) if
                             i.lower().__contains__('bopo')].pop() 
        ge_cast_header = [i for i in list(df_nutrient.columns) if
                          i.lower().__contains__('cast')].pop()
        btl_cast_header = [i for i in list(df_btl.columns) if
                           i.lower().__contains__('cast')].pop()
        ge_jointField = [ge_time_header, ge_station_header, ge_bottle_header, ge_cast_header]
        btl_jointField = [btl_time_header, btl_station_header, btl_bottle_header, btl_cast_header]
        
        #####################################
        df_nutrient_header = [h.split(' ')[0] for h in df_nutrient.columns]
        df_nutrient.columns = df_nutrient_header
        dfnutrient_to_merge = df_nutrient
        dfbtl_to_merge = df_btl
        dfbtl_to_merge = dfbtl_to_merge.drop(btl_time_header, axis=1)
        dfnutrient_to_merge = dfnutrient_to_merge.drop(ge_time_header, axis=1)
        dfnutrient_to_merge = dfnutrient_to_merge.drop(ge_station_header, axis=1)
        dfnutrient_to_merge[ge_time_header] = pd.to_datetime(df_nutrient[ge_time_header]).dt.strftime('%Y-%m-%d')
        # dfnutrient_to_merge[ge_time_header] = df_nutrient[ge_time_header].dt.strftime('%Y-%m-%d')
        u = []
        for i in df_nutrient[ge_station_header].values:
            if i.isdigit():
                u.append('G' + i)
            else:
                u.append(i)
        dfnutrient_to_merge[ge_station_header] = u
        hdf_time_units = "days since 1970-01-01 00:00:00"
        list_tmp = []
        import cftime
        for i in range(len(df_btl[btl_time_header])):
            u=num2date(df_btl[btl_time_header].values[i], hdf_time_units)
            u=cftime.DatetimeGregorian.strftime(u, '%Y-%m-%d')
            list_tmp.append(u)
        dfbtl_to_merge[btl_time_header] = list_tmp
        '''https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/'''
        df = pd.merge(dfnutrient_to_merge, dfbtl_to_merge, how="inner", left_on=ge_jointField, right_on=btl_jointField)
        df.to_csv(file_ge_btl, header=1, index=0)
        del df
        return 0


class merging_gui_jupiter():
    # This is the class where the GUI is made
    def __init__(self):
        self.gridwindow={} # making an empty grid window
        self.vbox_widgets = [] # making an empty vertical box
        self.gridwindow['grid'] = widgets.GridspecLayout(1, 1)
        #####
        self.getBTLbutton = widgets.Button(description="retrieve", layout=widgets.Layout(width='max-content'), button_style='info')
        self.getNutrientbutton = widgets.Button(description="retrieve", layout=widgets.Layout(width='max-content'), button_style='info')
        self.Continuebutton = widgets.Button(description="Continue", layout=widgets.Layout(width='max-content'))
        # BTL file retrieval
        layout = widgets.Layout(height='auto', width='125px')
        printmd('<h1><b>Merging bottle file with the nutrient file</b></h1>')
        # read text
        f=open("md_texts/nutrient_btl_infos.md","r")
        fc=f.read()
        f.close()
        text_html1 = markdown.markdown(fc)
        del fc
        self.gridwindow['text1'] = widgets.HTML(text_html1)
#         # Reading the images of the CTD-Rosette ##########
        ctd_img = open("images/ctd-rosette.jpg", "rb")
        ship_img = open("images/CCGSAmundsen.png", 'rb')
        ctd = ctd_img.read()
        ship = ship_img.read()
        gridimage1 = widgets.Image(value=ship, format='jpg', width=300)
        Figure1 = widgets.Label(r'\(\textbf{Figure 1:}\)'+' Canadian Coast Guard Ship ' + r'\(\textit{Amundsen}\)', layout=widgets.Layout(height='auto', width='auto'))
        gridimage2 = widgets.Image(value=ctd, format='png', width=300)
        Figure2 = widgets.Label(r'\(\textbf{Figure 2:}\)'+' CTD-Rosette', layout=widgets.Layout(height='auto', width='auto'))
        ship_img.close()
        ctd_img.close()
        image_vbox1 = widgets.VBox(children=[gridimage1, Figure1])
        image_vbox2 = widgets.VBox(children=[gridimage2, Figure2])
        self.gridwindow['image'] = widgets.HBox(children=[image_vbox1, image_vbox2])
# #         image_vbox = [[gridimage1, gridimage2], ['CCGS Amundsen', 'CTD-Rosette']]
#         gridwindow['image'] = widgets.HBox(children=[gridimage2, gridimage1])
        ###################################################
        self.vbox_widgets.append(self.gridwindow['text1'])
        self.vbox_widgets.append(self.gridwindow['image'])
        
                # read text
        f=open("md_texts/data_retrieval.md","r")
        fc=f.read()
        f.close()
        data_retrieval = markdown.markdown(fc)
        del fc
        self.gridwindow['data_retrieval'] = widgets.HTML(data_retrieval)
        self.vbox_widgets.append(self.gridwindow['data_retrieval'])
        ## Bottle file retrieval ######
        label = widgets.Label('Bottle files', layout=layout)
        self.BottleData = widgets.Text(
            value="http://jorvik.uwaterloo.ca:8080/opendap/data/CCADI/Amundsen_BTL_GreenEdge2016_LEG1.h5",
            layout=widgets.Layout(width='50%')
        )
        self.gridwindow['bottle'] = widgets.HBox(children=[label, self.BottleData, self.getBTLbutton])
        self.vbox_widgets.append(self.gridwindow['bottle'])
        label = widgets.Label('Nutrient file', layout=layout)
        self.nutrientServer = widgets.Text(
            value="https://CanWINerddap.ad.umanitoba.ca/erddap",
            layout=widgets.Layout(width='50%')
        )
        self.gridwindow['nutrientserver'] = widgets.HBox(children=[label, self.nutrientServer, self.getNutrientbutton])
        self.vbox_widgets.append(self.gridwindow['nutrientserver'])
        
        self.list0 = widgets.SelectMultiple(
            options=["Empty"],
            value=["Empty"],
            disabled=False
        )

        self.list1 = widgets.SelectMultiple(
            options=["Empty"],
            value=["Empty"],
            disabled=False
        )
        
        self.depthRange = widgets.FloatRangeSlider(
                value=[0, 0],
                min=0,
                max=5000,
                step=0.1,
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
        )
        
        self.outputdir = "2016_int_btl_csv"
        self.gridwindow['grid'][0, 0] = widgets.VBox(children=self.vbox_widgets)  # pass all the content of the vertical box into the left side of the grid
        
        self.getBTLbutton.on_click(self.getBTLdata)
        self.getNutrientbutton.on_click(self.getNutrientdata)
        display(self.gridwindow['grid'])
        self.Merge_Button=widgets.Button(
            description='Merge',
            disabled=False,
            button_style='', 
            tooltip='Click me',
            icon=''
            )
        self.merge_btl_nutrient()
        
        
        ###########
             
    def continue_to_pyco2sys(self): 
        def on_button_pyco2sys(b):
            continueprocess()

        outmerge=widgets.Output()
        @outmerge.capture()
        def continueprocess():
            checkInputfile()
#             self.continue_to_sic()
        
        gridwindow={}
        vbox_widgets = []
        gridwindow['grid'] = widgets.GridspecLayout(1,1)
        layout = widgets.Layout(height='auto', width='125px')
        f=open("md_texts/variable_Join_list.md","r")
        fc=f.read()
        f.close()
        text_var_sel = markdown.markdown(fc)
        del fc
        gridwindow['variable_selection'] = widgets.HTML(text_var_sel)
        vbox_widgets.append(gridwindow['variable_selection'])
        label = widgets.Label('Bottle_variables', layout=widgets.Layout(width='50%'))
        label = widgets.Label('Nutrient variables', layout=widgets.Layout(width='50%'))
        gridwindow['bottle variable list'] = widgets.HBox(children=[label, self.list0])

        gridwindow['nutrient variable list'] = widgets.HBox(children=[label, self.list1])
        gridwindow['var_list'] = widgets.HBox(children=[gridwindow['bottle variable list'], gridwindow['nutrient variable list']])
        vbox_widgets.append(gridwindow['var_list'])
        label = widgets.Label('Sample depth:', layout=layout)
        # read text
        f=open("md_texts/variable_meaning.md","r")
        fc=f.read()
        f.close()
        text_var = markdown.markdown(fc)
        del fc
        gridwindow['variable_meaning'] = widgets.HTML(text_var)
        # read text
        f=open("md_texts/sample_depth_range.md","r")
        fc=f.read()
        f.close()
        text_var_sel = markdown.markdown(fc)
        del fc
        gridwindow['sample_depth_range'] = widgets.HTML(text_var_sel)
        vbox_widgets.append(gridwindow['sample_depth_range'])
        gridwindow['Sample depth'] = widgets.HBox(children=[label, self.depthRange])
        vbox_widgets.append(gridwindow['Sample depth'])

        self.btl = pd.DataFrame()
        self.nutrient=pd.DataFrame()
        
        continue_button1=widgets.Button(
            description='Continue',
            disabled=False,
            button_style='', 
            tooltip='Click me',
            icon=''
            )
        gridwindow['merge'] = widgets.HBox(children=[self.Merge_Button])
        gridwindow['to_pyco2sys'] = widgets.HBox(children=[gridwindow['merge'], continue_button1])
        vbox_widgets.append(gridwindow['to_pyco2sys'])
        gridwindow['grid'][0, 0] = widgets.VBox(children=vbox_widgets)
        self.Merge_Button.on_click(self.clickMerge)
        continue_button1.on_click(on_button_pyco2sys)
        display(gridwindow['grid'])
        display(outmerge)
        return 0       


        
    def merge_btl_nutrient(self): 
        def on_button_continuemerge(b):
            continuemerge()

        out=widgets.Output()
        @out.capture()
        def continuemerge():           
            ###### transit to the PyCO2SYS #####
            self.continue_to_pyco2sys()
            

        self.continue_button=widgets.Button(
            description='continue',
            disabled=False,
            button_style='', 
            tooltip='Click me',
            icon=''
            )
        self.gridwindow['continue'] = widgets.HBox(children=[self.continue_button])
        self.vbox_widgets.append(self.gridwindow['continue'])
        self.gridwindow['grid'][0, 0] = widgets.VBox(children=self.vbox_widgets)
        self.continue_button.on_click(on_button_continuemerge)
        display(out)

    def getBTLdata(self,a):
        if not os.path.exists(self.outputdir):
            os.makedirs(self.outputdir)
        merge_btl_nutrient().get_btlfile(self.BottleData.value, self.outputdir)
        df_btl=merge_btl_nutrient().btl_to_dataframe(self.outputdir)
        self.list0.options=tuple(df_btl.columns)
        self.list0.value=["BOPO","CTDTmp90","Cast_Number","P_sal_CTD","Pres_Z","depth","latitude","longitude","time","station"]
        self.list0.rows = 24
        self.depthRange.min = df_btl['depth'].min()
        self.depthRange.max = df_btl['depth'].max()
        self.depthRange.value = [self.depthRange.min, self.depthRange.max]
        self.getBTLbutton.description="Success!"
        self.getBTLbutton.button_style='success'
        del df_btl
        return 0
    
    def getNutrientdata(self, a):
        if not os.path.exists(self.outputdir):
            os.makedirs(self.outputdir)
        e_DataSearch = ERDDAP(server=self.nutrientServer.value)
        result_search = e_DataSearch.get_search_url(search_for="greenedge", response="csv")
        self.datasetID = [k 
                          for k in pd.read_csv(result_search)["Dataset ID"] 
                          if k.lower().__contains__("greenedge_nutrient")].pop()
        #self.datasetID = pd.read_csv(result_search)["Dataset ID"][0]
        #print(result_search)

        e_datafetch = ERDDAP(server=self.nutrientServer.value, protocol="tabledap", response="csv")
        e_datafetch.dataset_id = self.datasetID

        df_nutrient = e_datafetch.to_pandas(parse_dates=True)
        file_ge = os.path.join(self.outputdir, f'{self.datasetID}.csv')  ## Nutrient file name 
        df_nutrient.to_csv(file_ge, index=False, header=True)
        self.list1.options=df_nutrient.columns
        """ Adjustment done in order to look easily for the variables needed in the data field"""
        station = [k for k in df_nutrient.columns if k.lower().__contains__("station")].pop()
        sample_date = [k for k in df_nutrient.columns if k.lower().__contains__("sample_date")].pop()
        #sample_date = sample_date.split(" ")[0]
        sample_depth = [k for k in df_nutrient.columns if k.lower().__contains__("sample_depth")].pop()
        #sample_depth = sample_depth.split(" ")[0]
        cast = [k for k in df_nutrient.columns if k.lower().__contains__("cast")].pop()
        bottle = [k for k in df_nutrient.columns if k.lower().__contains__("bottle")].pop()
        dic_um = [k for k in df_nutrient.columns if k.lower().__contains__("dic_um")].pop()
        #dic_um = dic_um.split(" ")[0]
        totalk = [k for k in df_nutrient.columns if k.lower().__contains__("totalk_l_um")].pop()
        #totalk = totalk.split(" ")[0]
        self.list1.value=[station, sample_date, sample_depth, cast, bottle,dic_um, totalk]
        self.list1.rows = 24
        self.getNutrientbutton.description="Success!"
        self.getNutrientbutton.button_style='success'
        del df_nutrient
        return 0

        
    def clickMerge(self, a):
        file_ge_btl = os.path.join(self.outputdir, 'merged_btl_nutrient.csv') ## Merged file name to be fed to the PyCO2SYS
        objectsForMerging = merge_btl_nutrient()
        if os.path.exists(file_ge_btl):
            os.remove(file_ge_btl)
            df_btl = objectsForMerging.btl_to_dataframe(self.outputdir)
            df_btl.reset_index(drop=True, inplace=True)
            cond = (df_btl["depth"].values[:]>=self.depthRange.value[0]) & (df_btl["depth"].values[:]<=self.depthRange.value[1])
            df_btl = df_btl.loc[cond]
            df_nutrient = pd.read_csv(os.path.join(self.outputdir, f'{self.datasetID}.csv'), header=0)            
            objectsForMerging.merge(df_nutrient=df_nutrient[list(self.list1.value)], df_btl=df_btl[list(self.list0.value)], file_ge_btl=file_ge_btl)
            del df_nutrient, df_btl
        else:
            df_btl = objectsForMerging.btl_to_dataframe(self.outputdir)
            df_btl.reset_index(drop=True, inplace=True)
            cond = (df_btl["depth"].values[:]>=self.depthRange.value[0]) & (df_btl["depth"].values[:]<=self.depthRange.value[1])
            df_btl = df_btl.loc[cond]
            df_nutrient = pd.read_csv(os.path.join(self.outputdir, f'{self.datasetID}.csv'), header=0)            
            objectsForMerging.merge(df_nutrient=df_nutrient[list(self.list1.value)], df_btl=df_btl[list(self.list0.value)], file_ge_btl=file_ge_btl)
            del df_nutrient, df_btl
        self.Merge_Button.description="Done"
        self.Merge_Button.button_style="success"
        return 0



In [ ]:
#Define the required parameters

def checkInputfile():
    #Create an empty list for all the required parameters in the file. 
    req_param_inFile=[]
    opt_param_inFile=[]
    
    pd.set_option('display.max_columns', 50)
    
    #-------------------INPUT FILE-------------------------------------------
    input_file =os.path.join("2016_int_btl_csv", "merged_btl_nutrient.csv")                
    df = pd.read_csv(input_file)
    df=df.reset_index(drop=True)
    #------------------------------------------------------------------------
    
    #Checking for the stadardized names in the input file to automatically pull out all the required, optional and mandatory parameters present
    
    # 1. KEY PARAMETERS 
    # Parameters- Total Alkalinity, DIC, PH, PCO2, fCO2, CO232, biCO2
    
    standardizedNames=['TotAlk_l_um_l','DIC_um_l','pH','pCO2','fCO2','CO232','biCO2'] #BODC standardized names
    fullNames=['Total alkalinity (umolkg1)','Dissolved inorganic carbon (umolkg1)','Partial pressure of carbon dioxide (pCO2) (uatm)',\
               'Fugacity of carbon dioxide (fCo2) (uatm)','Carbonate ion concentration (CO32) (umolkg1)','Bicarbonate ion (umol kg1)']   # Full name that will show up in widget
    
    
    for name, fname in zip(standardizedNames, fullNames): 
        if name in df.columns:
            req_param_inFile.append(fname) #Append the names of all the key parameters in the input file
        
    
    # 2. OPTIONAL PARAMETERS 
    # Parameters- SiOx, PO4, Ammonia, Sulfide
    
    standardizedNames=['SiOx_um_l','PO4_Filt_um_l','Ammonia','Sulfide'] #BODC standardized names
    fullNames=['Total Silicate (umolkg1)','Total Phosphate (SRP) (umolkg1)','Total Ammonia (umolkg1)','Total Sulfide (umolkg1)']   # Full name that will show up in widget
    
    for name, fname in zip(standardizedNames, fullNames): 
        if name in df.columns:
            opt_param_inFile.append(fname) #Append the names of all the key parameters in the input file
    
    
    getUserParameters(df, req_param_inFile, opt_param_inFile) 


In [ ]:
def getUserParameters(df,req_param_inFile,opt_param_inFile):

    #The user will be presented with the parameters automatically pulled out from the input file. They will have a chance to make changes to the selections. 
    #Only exception is if there are only two req parameters in input file, they will not be able to make any changes/selctions in this case.
    
    #Key Parameters Widget
    req_param_user=widgets.SelectMultiple(
        options=req_param_inFile,
        #value=req_param_inFile,
        #description='Key Parameters:',
        disabled=False,
    )
    req_param_user.layout.margin='0.5% 0% 5% 0%'
    req_param_user.layout.width='20%'
    req_param_user.layout.height='70%'


    #Optional Parameters Widget
    opt_param_user=widgets.SelectMultiple(
        options=opt_param_inFile,
        value=opt_param_inFile,
        #description='Optional Parameters:',
        disabled=False,
    )
    opt_param_user.layout.margin='0.5% 0% 3% 0%'
    opt_param_user.layout.width='20%'

    cont_button1=widgets.Button(
    description='Continue',
    disabled=False,
    button_style='', 
    tooltip='Click me',
    icon=''
    )

    
    # Onclick function for the first Continue button widget
    output = widgets.Output()
    @output.capture()
    def on_button_clicked(b):
        getConstants(df, req_param_user, opt_param_user, req_param_inFile, opt_param_inFile)
    

    # Key parameters, aka carbonate system parameters
    printmd('### <br><br/> Carbonate System Parameters ###') 
    printmd('More information on these arguments an be found [here](https://pyco2sys.readthedocs.io/en/latest/co2sys_nd/#carbonate-system-parameters).')
    #if there is only one or no key parameters in input file
    if len(req_param_inFile)<2:
        printmd("<br>**There are not enough key parameters for calculation of the full carbonate system. Please check input file and try again.**<br />")
        sys.exit(-1)
    
    #If only two key parameters in the input file, automatically use those two
    if len(req_param_inFile)==2:
        printmd("<br>**The following key carbonate parameters were found in the input file and will be used in calculations.**<br />")

        for name in req_param_inFile:
            printmd('- {}'.format(name))
 
    #If there are more than two key parameters in the input file, ask user to select any two
    if len(req_param_inFile)>2:
        printmd("<br>**The following key carbonate parameters were found in the input file. Choose any two parameters.**<br />")
        display(req_param_user) #display widget


    # Optional parameters, aka Nutrients & solutes
    printmd('### <br><br/> Nutrients and other solutes ###') 
    printmd('More information on these arguments an be found [here](https://pyco2sys.readthedocs.io/en/latest/co2sys_nd/#nutrients-and-other-solutes).')
    
    # If there is at least one opt parameter in file, display them and ask user to select any of them. All are automatically selected in the widget
    if len(opt_param_inFile)>0:
        printmd("<br>**The following nutrient parameters are in the input file. Choose any parameter(s).**</b>")
        display(opt_param_user)   #display widget
    
    
    display(cont_button1) #display continue button
    cont_button1.on_click(on_button_clicked)  #Call onclick function
    display(output) #display widget ouput when button is clicked
    

In [ ]:
def getConstants(df, req_param_user, opt_param_user, req_param_inFile, opt_param_inFile):    
    
    # Constants
    printmd('### <br><br/> Settings for constants ###') 
    printmd('More information on these constants an be found [here](https://pyco2sys.readthedocs.io/en/latest/co2sys_nd/#settings). Default constants chosen based on [Jiang et al., 2022](https://www.frontiersin.org/articles/10.3389/fmars.2021.705638/full).')
    #Widgets for the different constants
    phstr = widgets.Output()
    @phstr.capture()
    def constStrings1():
        printmd("<br>**Choose the pH scale:**")
        
    constStrings1()

    option_list=['1. Total',
                 '2. Seawater',
                 '3. Free',
                 '4. NBS, i.e. relative to NBS/NIST reference standards']
    
    #PH Scale
    phscale=widgets.RadioButtons(
        options=option_list,  
        disabled=False,
        layout={'width': 'max-content'},
    )
    phscale.layout.margin='0.5% 1% 3% 0%'
    #phscale.layout.width='40%' 


    # Carbonic Acid Dissociation
    k1k2str = widgets.Output()
    @k1k2str.capture()
    def constStrings2():
        printmd("**Choose the set of equilibrium constant parameterisations to model carbonic acid dissociation:**")
    constStrings2()

    option_list=['1. RRV93 (0 < T < 45 °C, 5 < S < 45, Total scale, artificial seawater).',
                '2. GP89 (−1 < T < 40 °C, 10 < S < 50, Seawater scale, artificial seawater).',
                '3. H73a and H73b refit by DM87 (2 < T < 35 °C, 20 < S < 40, Seawater scale, artificial seawater).',
                '4. MCHP73 refit by DM87 (2 < T < 35 °C, 20 < S < 40, Seawater scale, real seawater).',
                '5. H73a, H73b and MCHP73 refit by DM87(2 < T < 35 °C, 20 < S < 40, Seawater scale, real seawater)',
                '6. MCHP73 aka "GEOSECS" (2 < T < 35 °C, 19 < S < 43, NBS scale, real seawater).',
                '7. MCHP73 without certain species aka "Peng" (2 < T < 35 °C, 19 < S < 43, NBS scale, real seawater).',
                '8. M79 (0 < T < 50 °C, S = 0, freshwater only).',            
                '9. CW98 (2 < T < 30 °C, 0 < S < 40, NBS scale, real estuarine seawater).',
                '10. LDK00 (2 < T < 35 °C, 19 < S < 43, Total scale, real seawater).',
                '11. MM02 (0 < T < 45 °C, 5 < S < 42, Seawater scale, real seawater).',
                '12. MPL02 (−1.6 < T < 35 °C, 34 < S < 37, Seawater scale, field measurements).',
                '13. MGH06 (0 < T < 50 °C, 1 < S < 50, Seawater scale, real seawater).',
                '14. M10 (0 < T < 50 °C, 1 < S < 50, Seawater scale, real seawater).',
                '15. WMW14 (0 < T < 45 °C, 0 < S < 45, Seawater scale, real seawater).',
                '16. SLH20 (−1.67 < T < 31.80 °C, 30.73 < S < 37.57, Total scale, field measurements).',
                '17. SB21 (15 < T < 35 °C, 19.6 < S < 41, Total scale, real seawater).']
    
    k1k2=widgets.RadioButtons(
        options=option_list,    
        value='10. LDK00 (2 < T < 35 °C, 19 < S < 43, Total scale, real seawater).',
        #rows=len(option_list),
        layout={'width': 'max-content'},
        disabled=False,
    )
    k1k2.layout.margin='0.5% 1% 3% 0%'
    #k1k2.layout.width='50%'
    #k1k2.layout.height='100%'

    # Bisulfate ion dissociation 
    kso4str = widgets.Output()
    @kso4str.capture()
    def constStrings3():
        printmd("**Choose the equilibrium constant parameterisations to model bisulfate ion dissociation:**")
    constStrings3()

    option_list=['1. D90a: Dickson (1990) J. Chem. Thermodyn.',
                '2. KRCB77: Khoo et al. (1977) Anal. Chem.',
                '3. WM13: Waters & Millero (2013) Mar. Chem./ WMW14: Waters et al. (2014) Mar. Chem.']
    
    kso4=widgets.RadioButtons(
        options=option_list,    
        value='1. D90a: Dickson (1990) J. Chem. Thermodyn.',
        layout={'width': 'max-content'},
        #description='Parameter:',
        disabled=False,
    )
    kso4.layout.margin='0.5% 1% 3% 0%'
    #kso4.layout.width='40%'

    # Total borate 
    bostr = widgets.Output()
    @bostr.capture()
    def constStrings4():
        printmd("**Choose which boron:salinity relationship to use to estimate total borate:**")
    constStrings4()

    option_list=['1. U74: Uppström (1974) DeepSea Res.',
                '2. LKB10: Lee et al. (2010) Geochim. Cosmochim. Acta']
    
    bo=widgets.RadioButtons(
        options=option_list,    
        value='2. LKB10: Lee et al. (2010) Geochim. Cosmochim. Acta',
        layout={'width': 'max-content'},
        disabled=False,
    )
    bo.layout.margin='0.5% 1% 3% 0%'
    # bo.layout.width='40%'
    
    # hydrogen fluoride dissociation
    hfstr = widgets.Output()
    @hfstr.capture()
    def constStrings5():
        printmd("**Choose which which equilibrium constant parameterisation to use for hydrogen fluoride dissociation:**")
    constStrings5()

    option_list=['1. DR79: Dickson & Riley (1979) Mar. Chem.',
                '2. PF87: Perez & Fraga (1987) Mar. Chem.']
    
    hf=widgets.RadioButtons(
        options=option_list,    
        value='2. PF87: Perez & Fraga (1987) Mar. Chem.',
        layout={'width': 'max-content'},
        disabled=False,
    )
    hf.layout.margin='0.5% 1% 3% 0%'
    # hf.layout.width='40%' 

    # opt_gas_constant
    gcstr = widgets.Output()
    @gcstr.capture()
    def constStrings6():
        printmd("**Choose which value to use for the gas constant:**")
    constStrings6()
    option_list=['1. DOEv2',
                '2. DOEv3',
                '3. 2018 CODATA']
    
    gc=widgets.RadioButtons(
        options=option_list,    
        value='3. 2018 CODATA',
        layout={'width': 'max-content'},
        disabled=False,
    )
    gc.layout.margin='0.5% 1% 3% 0%'
    # gc.layout.width='40%' 
    
    #Continue button On-click function
    output = widgets.Output()
    @output.capture()
    def on_button_clicked(b):        
        runPyco2sys(df,req_param_user, opt_param_user, req_param_inFile, opt_param_inFile, phscale, k1k2, kso4, bo, hf,gc)

    # Button widget
    button2=widgets.Button(
    description='Continue',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon=''
    )

    box2 = widgets.VBox([phstr,phscale, k1k2str, k1k2, kso4str, kso4, bostr, bo, hfstr, hf, gcstr,gc, button2])
    display(box2)

    button2.on_click(on_button_clicked)
    display(output)

In [ ]:
def runPyco2sys(df,req_param_user, opt_param_user, req_param_inFile, opt_param_inFile, phscale, k1k2, kso4, bo, hf,gc):


    #Build the argument list for pyco2sys to ingest
    #The argument names are defined in the PyCO2 sys documentation.
    kwargs={}
   

    #KEY PARAMETERS
    
    # pyco2sys labels the two key parameters chosen as par 1 and par 2. 'par1check' checks if par1 has yet been assigned, otherwise it sets a key parameter to par 2
    par1check="False"
    par2check="False"

    
    #Check if the parameter was chosen by the user, or if it was automatically selected (only two parameters were in the file). In that case the variable would not be in 'req_param_user' from the widget.
    #If the varibale is to be used in the calculation, then get the data from the input file (saved as the data frame df)
    #The variable is then added to the arguments for pyco2sys
    
    substrings=['alkalinity','Dissolved inorganic carbon','pH','Partial pressure of carbon dioxide', 'Fugacity of carbon dioxide','Carbonate ion concentration', 'Bicarbonate Ion']
    standardizedNames=['TotAlk_l_um_l','DIC_um_l','pH','pCO2','fCO2','CO232','biCO2']
    types=[1,2,3,4,5,6,7]   #These are the different types according to pyco2sys documentation 
    
    
    for name, substr, t in zip(standardizedNames, substrings, types):           # Loop through the substrings and the standardized names  
        if par1check=="False" or par2check=="False":                            #Check if either par1 or par2 is false. We need at least two key paramaters (par1 and par2 represent these two parameters)
            if any(substr in string for string in req_param_inFile):            #If it is in the input file
                if len(req_param_inFile)==2:                                    #If it is one of only two key variables in the input file (here it would not be in 'req_param_user.value' as it would be automatically selected- no widget used)
   
                    if par1check=="True":        
                        kwargs['par2']=df[name].to_numpy(dtype=float)            #Get the data using the standardized name from the data frame (input file)
                        kwargs['par2_type']=t
                        par2check="True"
                    else:
                        kwargs['par1']=df[name].to_numpy(dtype=float)             
                        kwargs['par1_type']=t
                        par1check="True"

                elif len(req_param_inFile)>2:                                    # If there are more than two key parameters in the input file
                    if any(substr in string for string in req_param_user.value): #If there are more than two key variables in the file, then check if the user actually selected this varibale
                        if par1check=="True":        
                            kwargs['par2']=df[name].to_numpy(dtype=float)          
                            kwargs['par2_type']=t
                            par2check="True"
                        else:
                            kwargs['par1']=df[name].to_numpy(dtype=float)          
                            kwargs['par1_type']=t
                            par1check="True"



                            
    # OPTIONAL PARAMETERS
    
    substrings_opt=['Silicate','Phosphate','Ammonia','Sulfide' ]
    standardizedNames=['SiOx_um_l','PO4_Filt_um_l','Ammonia','Sulfide']
    pyco2sysNames=['total_silicate','total_phosphate','total_phosphate','total_sulfide']  # Names that pyco2sys expects in the argument list
    
    
    if len(opt_param_inFile)>0:  # if there is at least one optional paramter in the input file.        
        for name, substr, pName in zip(standardizedNames, substrings_opt, pyco2sysNames):     
            if any(substr in string for string in opt_param_user.value):  # If the optional parameter was chosen by the user
                kwargs[pName]=df[name].to_numpy(dtype=float)              # Get the data using the standardized name
        

        
    # MANDATORY  PARAMETERS

    # Also check if the user wanted to use any other output temperature and pressure in the calculations.
    # If they did not, the temp_out and press_out would be 'nan'
    
    #Set the output temperature and pressure to nan
    Temperature_out=float('nan')
    Pressure_out=float('nan')
    
    Temperature=df['CTDTmp90'].to_numpy(dtype=float)
    kwargs['temperature']=Temperature
    #if np.isnan(Temperature_out)==False:    
    kwargs['temperature_out']=Temperature_out

    Pressure=df['Pres_Z'].to_numpy(dtype=float)
    kwargs['pressure']=Pressure   
    #if np.isnan(Pressure_out)==False:    
    kwargs['temperature_out']=Pressure_out   

    Salinity=df['P_sal_CTD'].to_numpy(dtype=float)
    kwargs['salinity']=Salinity   



    #Get the values of the widget selections for the different CONSTANTS. All parameters are selected by default.
    
    k1k2Value=k1k2.value
    kso4Value=kso4.value
    boValue=bo.value
    hfValue=hf.value
    phscaleValue=phscale.value
    gcValue=gc.value

    kso4Value_temp=kso4Value[:2]
    boValue_temp=boValue[:2]
    hfValue_temp=hfValue[:2]
    k1k2Value_temp=k1k2Value[:2]
    phscaleValue_temp=phscaleValue[:2]
    gcValue_temp=gcValue[:2]
        
    if '.' in kso4Value_temp:      
        kso4Value_temp=kso4Value_temp[:1]
    
    if '.' in boValue_temp:       
        boValue_temp=boValue_temp[:1]       

    if '.' in hfValue_temp:       
        hfValue_temp=hfValue_temp[:1]       
        
    if '.' in k1k2Value_temp:       
        k1k2Value_temp=k1k2Value_temp[:1]
              
    if '.' in phscaleValue_temp:   
        phscaleValue_temp=phscaleValue_temp[:1]
        
    if '.' in gcValue_temp:
        gcValue_temp=gcValue_temp[:1]
    
    k1k2Value=int(k1k2Value_temp)
    kso4Value=int(kso4Value_temp)
    boValue=int(boValue_temp)
    hfValue=int(hfValue_temp)
    phscaleValue=int(phscaleValue_temp)
    gcValue=int(gcValue_temp)
        
    #Add them as arguments for pyco2sys
    kwargs['opt_k_carbonic']=k1k2Value
    kwargs['opt_k_bisulfate']=kso4Value
    kwargs['opt_total_borate']=boValue
    kwargs['opt_k_fluoride']=hfValue
    kwargs['opt_gas_constant']=gcValue


    # Import PyCO2SYS
    import PyCO2SYS as pyco2

    # Run pyCO2SYS!
    output_dict = pyco2.sys(**kwargs)

    
    Output(df,output_dict)

In [ ]:
def Output(df,output_dict):
    
#     ## Added to make the "continue button" to continue the process into the Sea Ice Concentration retrieval: 
    def on_button_sic(b):
        continue_sic()

    outsic=widgets.Output()
    @outsic.capture()
    def continue_sic():
        ccadi_uc3_mapping() # sea ice concentration processing

    ###########################################
    
    
    #The dictionary output_dict is uneven. Some elements are just a single int, string or float. The length of 'par1' will always be the longest length (it holds the first of the two key parameters from the input file)
    #For the values in the dictionary that are not single values, they are arrays. 
    #To be able to create even data frames, we need to seperate the values in the array and create a list.
    #Otherise, the array is saved as one value for each key, instad of a list of multiple values.
    
    
    longlength=len(output_dict['par1'])
    newlist=[]

    for val in output_dict.values():

        #if value is a single integer"
        if isinstance(val, int):
            list0=[val] * longlength
            newlist.append(list0)

        elif isinstance(val, str):
            list1=[val] * longlength
            newlist.append(list1)

        elif isinstance(val, float):
            list2=[val] * longlength
            newlist.append(list2)
        else:
            arr=val
            list3 = arr.tolist()
            newlist.append(list3)
            
            
    #Creae a new dict that has keys associated with a list of values, all of the same length.            
    newdict={}
    i=0
    for key in output_dict.keys():
        newdict[key]=newlist[i]
        i=i+1


    #Create a new dataframe and save as csv.
    output_df=pd.DataFrame.from_dict(newdict)

    #Merge this resulting data frame with extra variables from the input file that were not used in calculations
    cols_to_use = df.columns.difference(output_df.columns)   #variales that are different from those in output file
    input_subset=df[cols_to_use]
    merged_df = pd.concat([output_df,input_subset] , axis=1)  #Merged dataframe

        
    # Organize data frame so that specific varibales are at the front
    front_metadata=['project name','platform name','Cruise', 'Station','sample date','TIME','latitude','longitude','sample depth','Cast','Bottle']
    
    #Loop through the list of metadata variables that should be at the front
    col_position=-1
    for var in front_metadata: 
        for col in merged_df.columns:                                  # Loop through all the columns in dataframe
            stripped_string = re.sub("[^0-9a-zA-Z]+", " ",col)         # Strip the column headers of all non- laphanumeric characters
            if var.lower() in stripped_string.lower():                 # Check for column name regardless of case
                popped_col_data=merged_df.pop(col)                     # Pop the column from daat frame
                col_position=col_position+1                            # Find the next front position
                merged_df.insert(col_position, col, popped_col_data)   # Place variable at position

    front_data=['saturation_aragonite', 'saturation_aragonite_out','saturation_calcite','saturation_calcite_out','pCO2','fCO2','bicarbonate','pH_total']

    #Loop through the list of calculated carbonate chemistry variables that should be at the front
    for var in front_data: 
        for col in merged_df.columns:                                  # Loop through all the columns in dataframe
            if col==var:                 
                popped_col_data=merged_df.pop(col)                     # Pop the column from daat frame
                col_position=col_position+1                            # Find the next front position
                merged_df.insert(col_position, col, popped_col_data)   # Place variable at position
    
    
    #Remove par1, par2, par1 and par2 types- added by pyco2, not needed by user 
    merged_df.pop('par1')
    merged_df.pop('par2')
    merged_df.pop('par1_type')
    merged_df.pop('par2_type')

    #Remove duplicate columns from final data frame
    duplicateColumnNames = list()

    for x in range(merged_df.shape[1]):                   # Iterate over all the columns in dataframe
        col_name1= merged_df.columns[x]                   # Select column at xth index.

        for y in range(x + 1, merged_df.shape[1]):        # Iterate over all the columns in DataFrame from (x+1)th index till end
            col_name2= merged_df.columns[y]
            
            if col_name1.lower()==col_name2.lower():      # Check if column names are the same regardless of case
                duplicateColumnNames.append(col_name1)
                continue
                
            if '.1' in col_name1:                         # Check if there is a duplicate (same case), pandas will save this with a .1 at the end of the duplicated variable
                col_name1_stripped=col_name1.strip('.1')  # Remove .1 and check again for equality 
                if col_name1_stripped.lower()==col_name2.lower():
                    duplicateColumnNames.append(col_name1)
                    continue
                
            if '.1' in col_name2:
                col_name2_stripped=col_name2.strip('.1')
                if col_name1.lower()==col_name2_stripped.lower():
                     duplicateColumnNames.append(col_name2)
                        
    merged_df = merged_df.drop(columns=duplicateColumnNames)  #Drop all duplicates
    merged_df=merged_df.dropna(axis=1,how='all')              #Drop all empty columns

    # OUTPUT FILE----------------------------------------------------------------------
    if os.path.isfile(os.path.join("2016_int_btl_csv", "merged_btl_nutrient_pyco2sys.csv"))==True:  
        os.remove(os.path.join("2016_int_btl_csv", "merged_btl_nutrient_pyco2sys.csv"))
    merged_df.to_csv(os.path.join("2016_int_btl_csv", "merged_btl_nutrient_pyco2sys.csv"), index=False)   
    # OUTPUT FILE----------------------------------------------------------------------

    
    printmd('**<br />PCO2sys ran successfully! Output file is saved as merged_btl_nutrient_pyco2sys.csv**')
    printmd("<br />**Retrieving ice concentration now...**")
 
    # add the "continue button" on the GUI ######################################
    gridwindow={}
    vbox_widgets = []
    gridwindow['grid'] = widgets.GridspecLayout(1,1)

    continue_button2=widgets.Button(
        description='Continue',
        disabled=False,
        button_style='', 
        tooltip='Click me',
        icon=''
        )
    gridwindow['to_sic'] =  widgets.HBox(children=[continue_button2])
    vbox_widgets.append(gridwindow['to_sic'])
    gridwindow['grid'][0, 0] = widgets.VBox(children=vbox_widgets)
    display(gridwindow['grid'])
    continue_button2.on_click(on_button_sic)
    display(outsic)
    ##################################################

    

In [ ]:
L = mlp.Lock()
class addSeaIceConcentration:
    def __init__(self, inputfile):
        self.inputfile = inputfile

    def createShapefile(self, df, output_shp, time_header, prglabel):
        ''' This function creates a shapefile from the geographical locations inside the input CSV file.
            The coordinate system used for the output shapetile is the World Geodetic System (WGS) 1984.
            Both Latitude and Longitude are in decimal degrees'''
        prglabel.value = 'Creating the final shapefile...'
        shpfile = os.path.join(os.path.dirname(self.inputfile), output_shp, output_shp + '.shp')
        if not os.path.exists(os.path.join(os.path.dirname(self.inputfile), output_shp)):
            os.makedirs(os.path.join(os.path.dirname(self.inputfile), output_shp))
        driver = ogr.GetDriverByName("ESRI Shapefile")
        if os.path.exists(shpfile):
            driver.DeleteDataSource(shpfile)
        ds = driver.CreateDataSource(shpfile)
        spref = osr.SpatialReference()
        spref.ImportFromEPSG(4326)
        layer = ds.CreateLayer('StationsLocations', spref, ogr.wkbPoint)
        # create field to the layer
        for c in df.columns:
            u = list(df[c])
            vint = [i for i in u if isinstance(i, int)]
            vfloat = [i for i in u if isinstance(i, float)]
            if c.__contains__(time_header):
                layer.CreateField(ogr.FieldDefn(time_header, ogr.OFTDateTime))
            elif c.__contains__('CIS_dates'):
                layer.CreateField(ogr.FieldDefn('CIS_dates', ogr.OFTDateTime))
            elif c.__contains__('Time'):
                fieldname = ogr.FieldDefn('Time UTC', ogr.OFTString)
                fieldname.SetWidth(20)
                layer.CreateField(fieldname)
            elif len(u) == len(vint):
                df[c] = df[c].astype(float)
                ##############################################
                fieldname = ogr.FieldDefn(c, ogr.OFTInteger)
                fieldname.SetPrecision(0)
                layer.CreateField(fieldname)
            elif len(u) == len(vfloat):
                fieldname = ogr.FieldDefn(c, ogr.OFTReal)
                fieldname.SetPrecision(6)
                layer.CreateField(fieldname)
            else:
                df[c].astype(str)
                fieldname = ogr.FieldDefn(c, ogr.OFTString)
                fieldname.SetWidth(30)
                layer.CreateField(fieldname)

        c_fid = 0
        ld = layer.GetLayerDefn()
        lon = [i for i in df.columns if i.__contains__('longitude')].pop()
        lat = [i for i in df.columns if i.__contains__('latitude')].pop()
        for i in df.index:
            # create new point object
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(float(df[lon].values[i]), float(df[lat].values[i]))
            # create new feature
            featureDfn = layer.GetLayerDefn()
            feature = ogr.Feature(featureDfn)
            feature.SetGeometry(point)
            for k in range(0, df.columns.__len__()):
                fieldName = ld.GetFieldDefn(k).GetName()
                feature.SetField(fieldName, df[df.columns[k]].values[i])
            c_fid += 1
            # add the new feature to the new layer
            layer.CreateFeature(feature)
        del layer, ds, df
        return 0

    def extractFromUniBremenAMSR2(self, prm):
        ''' Sea Ice Concentration (SIC) from Bremen University are downloaded. The SIC located at each geographical
        coordinate indicated in the input csv file are extracted and then added as an extra-column to the csv file. '''
        L.acquire()
        ddate = pd.to_datetime(prm[0])
        dlat = prm[1]
        dlon = prm[2]
        q = prm[3]
        m = datetime.strftime(ddate, '%b').lower()  # extract the month in lowercase charachter
        y = datetime.strftime(ddate, '%Y')  # extract the year
        with tempfile.TemporaryDirectory() as tmpDir:
            url = r'https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/n6250/{0}/{1}/Arctic/asi-AMSR2-n6250-{2}-v5.4.tif'.format(
                y, m, datetime.strftime(ddate, '%Y%m%d'))
            if not os.path.exists(os.path.join(os.path.join(os.path.dirname(self.inputfile), 'BU_rasters'),
                                               'asi-AMSR2_{0}.tif'.format(
                                                   datetime.strftime(ddate, '%Y%m%d')))):
                urllib.request.urlretrieve(url, os.path.join(tmpDir, 'asi-AMSR2.tif'))
                if (q == 'Y'):
                    shutil.copy2(os.path.join(tmpDir, 'asi-AMSR2.tif'),
                                 os.path.join(os.path.join(os.path.dirname(self.inputfile), 'BU_rasters'),
                                              'asi-AMSR2_{0}.tif'.format(
                                                  datetime.strftime(ddate, '%Y%m%d'))))
                    src_filename = os.path.join(os.path.join(os.path.dirname(self.inputfile), 'BU_rasters'),
                                                'asi-AMSR2_{0}.tif'.format(
                                                    datetime.strftime(ddate, '%Y%m%d')))
                else:
                    src_filename = os.path.join(tmpDir, 'asi-AMSR2.tif')
            else:
                src_filename = os.path.join(os.path.join(os.path.dirname(self.inputfile), 'BU_rasters'),
                                            'asi-AMSR2_{0}.tif'.format(
                                                datetime.strftime(ddate, '%Y%m%d')))
            try:
                outval = self.pointExtract(src_filename, dlat, dlon)
                prm[4].put(prm[5])
            except:
                outval = np.nan
                prm[4].put(prm[5])
                pass
        L.release()
        return ([float(outval), datetime.strftime(ddate.to_pydatetime(), '%Y-%m-%d')])

    def extractFromCSI(self, prm):
        '''For each acquisition date in the csv file, differences are calculated between each one of them and each one included
        in the CIS tar files. Then the tar file corresponding the minimum difference is choosen for the extraction
        of the Sea Ice Concentration. '''
        L.acquire()  # this is needed to lock each process to let them running separately without writing in a same variable at the same time
        ddate = pd.to_datetime(prm[0])  # Acquisition date from the csv file.
        dlat = prm[1]
        dlon = prm[2]
        fcis = prm[3]
        CISRaster = prm[4]

        CIS_acquisition_times = [datetime.strptime(i.split('_')[2], '%Y%m%dT%H%MZ').date() for i in fcis]
        wq = np.array(CIS_acquisition_times)
        CIS_acquisition_times = list(np.unique(wq))
        csv_acquisition_time = datetime.strftime(ddate, '%Y%m%dT%H%M%S')
        sample_date = datetime.strptime(csv_acquisition_time, '%Y%m%dT%H%M%S').date()
        dt_abs = [abs(sample_date - each_date) for each_date in CIS_acquisition_times]
        closest_date = CIS_acquisition_times[dt_abs.index(min(dt_abs))]
        outraster = os.path.join(CISRaster, [i for i in fcis if i.__contains__(datetime.strftime(closest_date, '%Y%m%d'))][0][:-4] + '.tif')
        intval = self.pointExtract(outraster, dlat, dlon)
        outval = intval[0][0]
        prm[5].put(prm[6])
        L.release()  # this releases the locked process
        return ([float(outval), datetime.strftime(closest_date, '%Y-%m-%d')])
#         return ([float(outval), datetime.strftime(CIS_acquisition_times[s[0][0]], '%Y-%m-%d')])

    def selectCISFiles(self, prm):
        '''This function select the filenames from the CIS acquired at the nearest time as each one of the within the
        input csv file. The output list of files will be used to download them. '''
        L.acquire()
        ddate = pd.to_datetime(prm[0])
        fcislist = prm[3]
        CIS_acquisition_times = [datetime.strptime(i.split('_')[2], '%Y%m%dT%H%MZ').date() for i in fcislist]
        wq = np.array(CIS_acquisition_times)
        CIS_acquisition_times = list(np.unique(wq))
        u = datetime.strftime(ddate, '%Y%m%dT%H%M%S') # sample date
        sample_date = datetime.strptime(u, '%Y%m%dT%H%M%S').date()
        dt_abs = [abs(sample_date - each_date) for each_date in CIS_acquisition_times]
        closest_date = CIS_acquisition_times[dt_abs.index(min(dt_abs))]
        prm[5].put(prm[6])
        L.release()
        return [i for i in fcislist if i.__contains__(datetime.strftime(closest_date, '%Y%m%d'))][0]

    def binaryretrieve(self, j):
        '''The connection to the FTP server of the Canadian Ice Service is done in this function.
        All the spatial coordinate are assumed to be within the region Eastern_Arctic and in the same year 2016
        as the case of the GreenEdge data. '''
        L.acquire()
        i = j[0]
        shp_for_UC3 = j[1]
        hostname = 'sidads.colorado.edu'
        ftp = FTP(hostname)
        ftp.login(user='anonymous', passwd='')
        ## This should be changed to be dynamically change depending on the region (here: Eastern_Arctic) and the year.
        # The User Guide from the Canadian Ice Service (https://nsidc.org/data/G02171/versions/1?qt-data_set_tabs=3#qt-data_set_tabs) discribe all the possible region names
        ftp.cwd('/pub/DATASETS/NOAA/G02171/Eastern_Arctic/2016/') 
        if not os.path.exists(os.path.join(shp_for_UC3, i)):
            with open(os.path.join(shp_for_UC3, i), 'wb') as localfile:
                ftp.retrbinary('RETR ' + i, localfile.write, 1024)
        ftp.quit()
        j[2].put(j[0])
        L.release()
        return 0

    def fetchTarFromCIS(self, tarfile, shp_for_UC3, CISRaster, prglabel, pr, pStatus):
        ''' This function manage the file retrieval from the CIS and then call to another function to do the vector
        shapefiles into rasters. '''

        if not os.path.exists(shp_for_UC3):
            os.makedirs(shp_for_UC3)
        '''For now, we assume all the data in the csv file were acquired in the same year (2016) and from
        the same region (Eastern_Arctic) as the case of the GreenEdge data.'''
        pool = mlp.Pool(processes=mlp.cpu_count()-2)
        m = Manager()
        queue = m.Queue()
        tarfilelist = [[i, shp_for_UC3, queue] for i in tarfile]
        s = pool.map_async(self.binaryretrieve, tarfilelist)
        ##
        while True:
            if s.ready():
                break
            else:
                c1 = int(queue.qsize() * 100 / len(tarfilelist))
                pr.value=c1
                prglabel.value = 'Fetching files from the CIS server...'
                pStatus.value = f'{pr.value}%'
        ##
        del pool, s, queue, m
        # Start a new progressbar for the shapefile converstion
        m = Manager()
        queue = m.Queue()
        pr.value=0
        prglabel.value = 'Converting shapefiles into raster files...'
        pStatus.value = f'{pr.value}%'
        ###################################################################
        for f in tarfile:
            '''Here the *.shp file have already been extracted from the *.tar file, 
            so we only need to point to it as shp_filename'''
            shutil.unpack_archive(os.path.join(shp_for_UC3, f), shp_for_UC3, f[-3:])
            shp_filename = os.path.join(shp_for_UC3, f[:-4] + '.shp')
            outraster = os.path.join(CISRaster, f[:-4] + '.tif')
            if not os.path.exists(outraster):
                self.makeRasterFromSHP(shp_filename, outraster, 100)
            queue.put(1)
            c1 = int(queue.qsize() * 100 / len(tarfilelist))
            pr.value=c1
            pStatus.value = f'{pr.value}%'
        del m, queue
        return 0


    def makeRasterFromSHP(self, shp_filename, outraster, pxlsize):
        ''' This function handle the transformation of the vector shapefile format into rasters format.
        The ogr python binding package is used to read the vector shepefile before their transfmation into raster.
         Here only the Field CT holding the Sea Ice Concentration data are rasterized.
         If needed, other Field amoung all of those included in the shapefile can be added as an additional band
         into the output raster. '''
        shpfile = ogr.Open(shp_filename)
        layer = shpfile.GetLayer()
        xmin, xmax, ymin, ymax = layer.GetExtent()
        cols = int((xmax - xmin) / pxlsize)
        rows = int((ymax - ymin) / pxlsize)
        rdrive = gdal.GetDriverByName('GTiff')
        ds = rdrive.Create(outraster, cols, rows, 1, gdal.GDT_Byte)
        ds.SetGeoTransform([xmin, pxlsize, 0, ymax, 0, -pxlsize])
        gdal.RasterizeLayer(ds, [1], layer, options=['ATTRIBUTE=CT'])
        ds.SetProjection(layer.GetSpatialRef().ExportToPrettyWkt())
        ds.GetRasterBand(1).SetNoDataValue(0)
        del ds, rdrive, shpfile, layer
        return 0

    def pointExtract(self, src_filename, dlat, dlon):
        ''' The extraction process is handled inside this function.
        The GDAL python binding package is used here to read the raster files needed for the extraction.
        The pyproj package is used to bring the geographical coordinates from the input csv to match the spatial
         coordinate system of the rasters in order to make the extraction of the right collocated pixel with the csv data. '''
        src_ds = gdal.Open(src_filename)
        gt = src_ds.GetGeoTransform()
        band = src_ds.GetRasterBand(1)
        proj = osr.SpatialReference(wkt=src_ds.GetProjection())
        #####
        wgs84 = pyproj.CRS("EPSG:4326")
        rstProj = pyproj.CRS(proj.ExportToProj4())
        #####
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(float(dlat), float(dlon))  # to make sure the corrdinates are not in string format
        mx, my = pyproj.Transformer.from_proj(wgs84, rstProj).transform(point.GetX(), point.GetY())
        px = int((mx - gt[0]) / gt[1])  # x pixel
        py = int((my - gt[3]) / gt[5])  # y pixel
        intval = band.ReadAsArray(px, py, 1, 1)
        del band, src_ds, point, proj, gt, mx, my, px, py
        return intval

    def getCISTarFileList(self, dlist, prglabel, pr, pStatus):
        ''' This function extract the file list needed for the extraction. It uses the acquisition year
        in the input csv file to locate the same year used in the FTP data endpoint in order to select the
        files to be downloaded. '''
        dl = pd.to_datetime(dlist)
        csv_year = [datetime.strftime(s, '%Y') for s in dl]
        csv_year = np.unique(csv_year)
        fcis_gen = []
        fcis = []
        m = Manager()
        queue = m.Queue()
        for y in csv_year:
            hostname = 'sidads.colorado.edu'
            ftp = FTP(hostname)
            ftp.login(user='anonymous', passwd='')
            ''' There is more regions to be considered. Here the region Easter_Arctic is directly selected. '''
            # TODO: Find a way to make automatic selection of the region of interest regarding the spatial extent of coordinate in the csv file used
            ftp.cwd('/pub/DATASETS/NOAA/G02171/Eastern_Arctic/{0}/'.format(y))
            files = ftp.nlst()  # This extract all the files within the folder named with the YEAR in.
            ftp.quit()
            fcis_gen.append([i for i in files if i.__contains__('cis')])
        for i in fcis_gen:
            fcis = fcis + i
            queue.put(i)
            p = int(queue.qsize()*100/len(fcis_gen))
            pr.value=p
            prglabel.value = 'Building file list...'
            pStatus.value = f'{pr.value}%'
        del fcis_gen, m, queue
        return fcis

    def ExtractionPixelValues(self, param_to_extract, q, rstsource, prglabel, pr, pStatus):
        ''' * inputfile: Comma Separated Value (CSV) file with a header containning Date, Latitude, and Longitude.
            * param_to_extract: name of the parameter to be extracted from the raster. This will become the name of
            the new column that will be added to the initial csv file. Here it is about Sea_Ice_Concentration.
            The format of the Date in the csv file should be mm/dd/yyyy.
            The Latitude and Longitude Should be in full decimal format and their values are in the range [-180, 180].

            * The Extraction Process is don using parallel computing to accelerate the process. Parallel processing is
            very usefull here as the Extraction of the data coresponding to each rows are independent from each other.
        '''

        outfile = os.path.join(os.path.split(self.inputfile)[0],
                               os.path.split(self.inputfile)[1][:-4] + '_{0}_{1}.csv'.format(param_to_extract,
                                                                                             rstsource))
        if os.path.exists(outfile):
            os.remove(outfile)
        df = pd.read_csv(self.inputfile, header=0, sep=',', parse_dates=True, skiprows=[1])
        time_header=[i for i in df.columns if
                     ((i.__contains__('Date')) or (i.__contains__('date')) or
                      (i.__contains__('Time')) or (i.__contains__('time')))].pop()
        if rstsource == 'BU':
            m = Manager()
            queue = m.Queue()
            lon = [i for i in df.columns if i.__contains__('longitude')].pop()
            lat = [i for i in df.columns if i.__contains__('latitude')].pop()
            p = [[df[time_header][i], df[lat][i], df[lon][i], q, queue, i] for i in df.index]
            if q == 'Y':
                if not os.path.exists(os.path.join(os.path.dirname(self.inputfile), 'BU_rasters')):
                    os.makedirs(os.path.join(os.path.dirname(self.inputfile), 'BU_rasters'))
            pool = mlp.Pool(mlp.cpu_count() - 2)
            s = pool.map_async(self.extractFromUniBremenAMSR2, p)
            ##
            while True:
                if s.ready():
                    break
                else:
                    c1=int(queue.qsize()*100/len(p))
                    pr.value=c1 
                    pStatus.value = f'{pr.value}%'
            print(queue.qsize())
            ##
            a = np.array(s.get())
            del s
            b = np.transpose(a)
            df[param_to_extract] = b[0]
            df[time_header] = b[1]
            del pool, m, queue
        else:
            CISRaster = os.path.join(os.path.dirname(self.inputfile), 'CISraster')
            if not os.path.exists(CISRaster):
                os.makedirs(CISRaster)
            ''' Retrieval from the CIS server of filename list corresponding to each dataset in the csv file'''
            fcis = self.getCISTarFileList(df[time_header], prglabel, pr, pStatus)
            m = Manager()
            queue = m.Queue()
            pr.value=0
            ''' Selection of each shapefile with a closest acquisition time to each dataset of the csv file. '''
            lon = [i for i in df.columns if i.__contains__('longitude')].pop()
            lat = [i for i in df.columns if i.__contains__('latitude')].pop()
            p = [[df[time_header][i], df[lat][i], df[lon][i], fcis, CISRaster, queue, i] for i in df.index]
            pool = mlp.Pool(mlp.cpu_count() - 2)
            imglist = pool.map_async(self.selectCISFiles, p)
            prglabel.value = 'Selecting CIS file...'
            while True:
                if imglist.ready():
                    break
                else:
                    c1=int(queue.qsize()*100/len(p))
                    pr.value=c1
                    pStatus.value = f'{pr.value}%'
            imgarray0 = np.array(imglist.get())
            imgarray1 = np.unique(imgarray0)
            imglist0 = list(imgarray1)
            CIS_shp = os.path.join(os.path.dirname(self.inputfile), 'CIS_shp')
            if not os.path.exists(CIS_shp):
                os.makedirs(CIS_shp)
            del imgarray0, imgarray1, pool, m, queue
            '''Fetching the files from the remote server'''
            self.fetchTarFromCIS(imglist0, CIS_shp, CISRaster, prglabel, pr, pStatus)
            pool = mlp.Pool(mlp.cpu_count() - 2)
            m = Manager()
            queue = m.Queue()
            p = [[df[time_header][i], df[lat][i], df[lon][i], fcis, CISRaster, queue, i] for i in df.index]
            s = pool.map_async(self.extractFromCSI, p)
            ##
            while True:
                if s.ready():
                    break
                else:
                    c1=int(queue.qsize()*100/len(p))
                    pr.value=c1
                    prglabel.value = 'Extracting SIC from newly created rasters...'
                    pStatus.value = f'{pr.value}%'
            del imglist0
            ##
            a = np.array(s.get())
            del pool, m, queue
            b = np.transpose(a)
            df[param_to_extract] = b[0]
            df['CIS_dates'] = b[1]
            del s
            if q == 'N':
                shutil.rmtree(os.path.join(os.path.dirname(self.inputfile), 'CISraster'))
                shutil.rmtree(os.path.join(os.path.dirname(self.inputfile), 'CIS_shp'))
        df.to_csv(outfile,
                  sep=',', index=False, header=1)
        output_shp = os.path.split(self.inputfile)[1][:-4] + '_{0}_{1}_SHP'.format(param_to_extract, rstsource) # Name of the output shapefile
        self.createShapefile(df, output_shp, time_header, prglabel)
        prglabel.value = 'Processing Finished!!'
        pStatus.value = f'{pr.value}%'
        output_gpkg = os.path.split(self.inputfile)[1][:-4] + '_{0}_{1}_gpkg'.format(param_to_extract, rstsource) # Name of the output Geopackage (gpkg) file
        if not os.path.exists(os.path.join(os.path.dirname(self.inputfile), output_gpkg)):
            os.makedirs(os.path.join(os.path.dirname(self.inputfile), output_gpkg))
        gpkgfile = os.path.join(os.path.dirname(self.inputfile), output_gpkg, output_gpkg + '.gpkg')
        os.system(f'ogr2ogr -a_srs EPSG:4326 -oo X_POSSIBLE_NAMES=Lon* -oo Y_POSSIBLE_NAMES=Lat*  -f "GPKG" {gpkgfile} {outfile}') # Creates the gpkg file from the shapefile. This can be created directly from the csv.
        return output_shp

    def getSeaIceSource(self, workDir, r, gLocalCopy, prglabel, pr, pStatus):
        ''' The choice between the Sea Ice Data from the Canadian Sea Ice Service and Bremen University is handled
         in this function. When the choice is done, another function is called to take care of the process of
         extracting Sea Ice Concentration from the chosen data source. '''
        if not os.path.exists(self.inputfile):
            print("Input file not exists !")
            exit(-1)
        if r == 1:
            r = 'BU'
        elif r == 2:
            r = 'CIS'
        else:
            exit(-1)
        if gLocalCopy == 'y':
            gLocalCopy = 'Y'
        elif gLocalCopy == 'n':
            gLocalCopy = 'N'
        t = datetime.now()
        output_shp = self.ExtractionPixelValues('sea_ice_co', gLocalCopy, r, prglabel, pr, pStatus)
        print(datetime.now() - t)
        return output_shp

class ccadi_uc3_mapping():
    def __init__(self):
        ## initiate the grid to display the contents of the page ###
        self.gridwindow={}
        self.vbox_widgets = []
        self.gridwindow['grid'] = widgets.GridspecLayout(1,1)
                
        #####

         # read text
        f=open("md_texts/SeaIceConcentration.md","r")
        fc=f.read()
        f.close()
        text_html1 = markdown.markdown(fc)
        del fc
        self.gridwindow['InfoSIC'] = widgets.HTML(text_html1)
        self.vbox_widgets.append(self.gridwindow['InfoSIC'])
        # Fields
        self.wdField = widgets.Text(
            value=os.path.join("2016_int_btl_csv", "merged_btl_nutrient_pyco2sys.csv"),
            layout=widgets.Layout(width='max-content')
        )

        self.chkb1 = widgets.Checkbox(
            value=False,
            description='Canadian Ice Service',
            disabled=False,
            indent=False
        )
        self.chkb2 = widgets.Checkbox(
            value=False,
            description='Bremen University',
            disabled=False,
            indent=False
        )
        self.chkb3 = widgets.Checkbox(
            value=False,
            description='Keep a local copy of the raster images',
            disabled=False,
            indent=False
        )
        self.gridwindow['checkbox'] = widgets.VBox(children=[self.chkb1, self.chkb2, self.chkb3])
        self.vbox_widgets.append(self.gridwindow['checkbox'])
        self.prg = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            bar_style='success',
            style={'bar_color': 'green'},
            orientation='horizontal',
            layout=widgets.Layout(width='800px')
        )
        self.status = widgets.Label(value=f'{self.prg.value}%', layout=widgets.Layout(width='max-content'))
        self.prg_label = widgets.Label('', layout=widgets.Layout(width='max-content'))
        self.vbox_widgets.append(self.prg_label)
        self.gridwindow['progressbar'] = widgets.HBox(children=[self.prg, self.status])
        self.vbox_widgets.append(self.gridwindow['progressbar'])
        self.okButton = widgets.Button(description="OK")
        
        self.okButton.on_click(self.clickOkbutton)
        #####
        
        self.UC3_mapping()


    def check_checkBox(self):
        c = 0
        value = ''
        q = 'n'
        if self.chkb1.value==True:
            value = self.chkb1.description
            c = 2
        if self.chkb2.value==True:
            value = self.chkb2.description
            c = 1
        if (self.chkb3.value==True):
            q = 'y'
        return c, q


    def clickOkbutton(self, b):
        self.prg.value=0
        self.prg_label.value = 'Processing...'
        workDir = os.path.dirname(self.wdField.value)
        if not os.path.exists(workDir):
            os.makedirs(workDir)
        inputfile = self.wdField.value
        r, q = self.check_checkBox()
        output_shp = addSeaIceConcentration(inputfile).getSeaIceSource(workDir, r, q, self.prg_label, self.prg, self.status)
        

    def UC3_mapping(self): 
        def on_button_clicked(b):
            showmap()

        out=widgets.Output()
        @out.capture()
        def showmap():

            workDir=os.path.join("2016_int_btl_csv")
            shp=os.path.join(workDir, "merged_btl_nutrient_pyco2sys_sea_ice_co_CIS_SHP", "merged_btl_nutrient_pyco2sys_sea_ice_co_CIS_SHP.shp")
            data_full=gpd.read_file(shp)
                        
            # Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
            data_full['geoid'] = data_full.index.astype(str)

#             dataf_0m=data.loc[np.round(data["sample_dep"].values)==1]
            dataf_10m=data_full.loc[np.round(data_full["sample_dep"].values)==10]
            dataf_20m=data_full.loc[np.round(data_full["sample_dep"].values)==20]
            dataf_30m=data_full.loc[np.round(data_full["sample_dep"].values)==30]
            dataf_40m=data_full.loc[np.round(data_full["sample_dep"].values)==40]
            dataf_50m=data_full.loc[np.round(data_full["sample_dep"].values)==50]
            dataf_60m=data_full.loc[np.round(data_full["sample_dep"].values)==60]
            dataf_70m=data_full.loc[np.round(data_full["sample_dep"].values)==70]
            dataf_80m=data_full.loc[np.round(data_full["sample_dep"].values)==80]
            dataf_90m=data_full.loc[np.round(data_full["sample_dep"].values)==90]
            dataf_100m=data_full.loc[np.round(data_full["sample_dep"].values)==100]

            ###

            lonCent = (data_full.bounds.maxx + data_full.bounds.minx).mean()/2
            latCent = (data_full.bounds.maxy + data_full.bounds.miny).mean()/2
            # creating a map object
            m = leafmap.folium.Map(location=(latCent,lonCent), projections="epsg3575", zoom_start=6)
            #rst = os.path.join("2016_int_btl_csv","CISraster","cis_SGRDREA_20160606T1800Z_pl_a.tif")

            ###
            # Create the variable plot upon click on the stations on the map
            def chart_func(df, st): #new function
                chart_temp = alt.Chart(df).mark_line(color='red').transform_fold(
                    fold=['CTDTmp90', 'sample_dep'], 
                    as_=['variable', 'value']).encode(
                        x=alt.X('CTDTmp90:Q', 
                                axis=alt.Axis(title='Temperature (°C)', 
                                              titleColor='red'), 
                                scale=alt.Scale(domain=[df['CTDTmp90'].min(), 
                                                        df['CTDTmp90'].max()])),
                        y=alt.Y('sample_dep:Q',
                                axis=alt.Axis(title='Depth (m)'), 
                                scale=alt.Scale(reverse=True, 
                                                domain=[0, df['sample_dep'].max()])),
                        color=alt.value('red')
                )
                chart_sal=alt.Chart(df).mark_line(color='green').transform_fold(
                    fold=['P_sal_CTD', 'sample_dep'], 
                    as_=['variable', 'value']).encode(
                        x=alt.X('P_sal_CTD:Q', 
                                axis=alt.Axis(title='Salinity', 
                                              titleColor='green'), 
                                scale=alt.Scale(domain=[df['P_sal_CTD'].min(), 
                                                        df['P_sal_CTD'].max()])),
                        y=alt.Y('sample_dep:Q', 
                                axis=alt.Axis(title='Depth (m)'), 
                                scale=alt.Scale(reverse=True, 
                                                domain=[0, df['sample_dep'].max()])),
                        color=alt.value('green')
                )
                ufchart=alt.layer(chart_temp, chart_sal, 
                                  title=f"Vertical profil of Salinity and Temperature at Station: {st}", 
                                  width=400, height=400).resolve_scale(x='independent').configure_axisTop(titleColor='green').configure_axisBottom(titleColor='red').resolve_legend(color='independent')            
                return ufchart.to_json()
            
            # extract unique coordinates
            data_full=data_full.round({'latitude':3, 'longitude':3})
            df=data_full[['latitude', 'longitude']].drop_duplicates() # drop all duplicated coordinates and keep the row indexes
            u=[]
            for i in df.index:  # use the indexes (kept in the precedent lines) to build a new dataframe from df
                u.append(data_full.values[i])
            dg=pd.DataFrame(u, columns=data_full.columns)
            data_coord=dg[['station', 'latitude', 'longitude']]
            del dg
            full_profile = leafmap.folium.FeatureGroup(name="Full profiles")
            for i, st in zip(df.index, data_coord['station'].values[:]):
                ds0=data_full[
                    [
                        'sample_dep',
                        'P_sal_CTD', 
                        'station', 
                        'CTDTmp90', 
                        'latitude', 
                        'longitude']
                ].loc[
                    data_full["station"].values==st
                ]
                ds2=ds0.dropna().round({
                    "CTDTmp90":2, 
                    "P_sal_CTD":2, 
                    'latitude': 3, 
                    'longitude':3})
                chart=chart_func(ds0, st)
                pp=leafmap.folium.Popup(max_width=600).add_child(leafmap.folium.VegaLite(chart, width=600))
                full_profile.add_child(leafmap.folium.CircleMarker(
                    location=[data_full['latitude'].values[i], data_full['longitude'].values[i]], radius=6,
                    popup=pp,
                ))
            full_profile.add_to(m)
            
            # Select only needed columns           
            data_10m = dataf_10m[['geoid', 'P_sal_CTD', 'station', 'sample_dep', 'CTDTmp90', 'geometry']]
            
            # Add data near the sea surface: 10m
            leafmap.folium.features.GeoJson(dataf_10m,
                                            name='Data at 10m depth',
                                            style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                                            tooltip=leafmap.folium.features.GeoJsonTooltip(
                                                fields=[
                                                    'P_sal_CTD', 
                                                    'station', 
                                                    'sample_dep', 
                                                    'CTDTmp90'],
                                                aliases = [
                                                    'Practical salinity from CTD', 
                                                    'Station name', 
                                                    'sample depth (m)',
                                                    'Temperature from CTD (°C)'
                                                ],
                                                sticky=False)
                                           ).add_to(m)


            # Select only needed columns
            data_20m = dataf_20m[['geoid', 'P_sal_CTD', 'station', 'sample_dep', 'CTDTmp90', 'geometry']]
            
            # Add data near the sea surface: 20m
            leafmap.folium.features.GeoJson(dataf_20m,
                                            name='Data at 20m depth',
                                            style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                                            tooltip=leafmap.folium.features.GeoJsonTooltip(
                                                fields=[
                                                    'P_sal_CTD', 
                                                    'station', 
                                                    'sample_dep', 
                                                    'CTDTmp90'],
                                                aliases = [
                                                    'Practical salinity from CTD', 
                                                    'Station name', 
                                                    'sample depth (m)',
                                                    'Temperature from CTD (°C)'
                                                ],
                                                sticky=False)
                                           ).add_to(m)
            
            # Select only needed columns
            data_30m = dataf_30m[['geoid', 'P_sal_CTD', 'station', 'sample_dep', 'CTDTmp90', 'geometry']]
            
            # Add data near the sea surface: 30m
            leafmap.folium.features.GeoJson(dataf_30m,
                                            name='Data at 30m depth',
                                            style_function=lambda x: {
                                                'color':'transparent',
                                                'fillColor':'transparent',
                                                'weight':0
                                            },
                                            tooltip=leafmap.folium.features.GeoJsonTooltip(
                                                fields=[
                                                    'P_sal_CTD', 
                                                    'station', 
                                                    'sample_dep', 
                                                    'CTDTmp90'],
                                                aliases = [
                                                    'Practical salinity from CTD', 
                                                    'Station name', 
                                                    'sample depth (m)',
                                                    'Temperature from CTD (°C)'
                                                ],
                                                sticky=False)
                                           ).add_to(m)

            # Select only needed columns
            data_40m = dataf_40m[['geoid', 'P_sal_CTD', 'station', 'sample_dep', 'CTDTmp90', 'geometry']]
            
            # Add data near the sea surface: 40m
            leafmap.folium.features.GeoJson(dataf_40m,
                                            name='Data at 40m depth',
                                            style_function=lambda x: {
                                                'color':'transparent',
                                                'fillColor':'transparent',
                                                'weight':0
                                            },
                                            tooltip=leafmap.folium.features.GeoJsonTooltip(
                                                fields=[
                                                    'P_sal_CTD', 
                                                    'station', 
                                                    'sample_dep', 
                                                    'CTDTmp90'],
                                                aliases = [
                                                    'Practical salinity from CTD', 
                                                    'Station name', 
                                                    'sample depth (m)',
                                                    'Temperature from CTD (°C)'
                                                ],
                                                sticky=False)
                                           ).add_to(m)
#           # Select only needed columns
            data_50m = dataf_50m[['geoid', 'P_sal_CTD', 'station', 
                                  'sample_dep', 'CTDTmp90', 'geometry', 
                                  'latitude', 'longitude']]
    
            # Add data near the sea surface: 50m
            leafmap.folium.features.GeoJson(dataf_50m,
                                            name='Data at 50m depth',
                                            style_function=lambda x: {
                                                'color':'transparent',
                                                'fillColor':'transparent',
                                                'weight':0
                                            },
                                            tooltip=leafmap.folium.features.GeoJsonTooltip(
                                                fields=[
                                                    'P_sal_CTD', 
                                                    'station', 
                                                    'sample_dep', 
                                                    'CTDTmp90'],
                                                aliases = [
                                                    'Practical salinity from CTD', 
                                                    'Station name', 
                                                    'sample depth (m)',
                                                    'Temperature from CTD (°C)'
                                                ],
                                                sticky=False)
                                           ).add_to(m)

#             ######################################################################################################################

            leafmap.folium.LayerControl().add_to(m)
            display(m)
        
        self.showmap_button=widgets.Button(
                description='Show Map',
                disabled=False,
                button_style='', 
                tooltip='Click me',
                icon=''
                )
        self.gridwindow['ok_and_continue'] = widgets.HBox(children=[self.okButton, self.showmap_button])
        self.vbox_widgets.append(self.gridwindow['ok_and_continue'])
        
        self.gridwindow['grid'][0, 0] = widgets.VBox(children=self.vbox_widgets)  #
        
        self.accordion0 = widgets.Accordion(
            children=[widgets.HBox(children = [self.gridwindow['grid'][0, 0]])]
        )
        self.accordion0.set_title(0, 'Adding Sea Ice Concentrations into the combined BTL_Nutrient file.')
        display(self.accordion0)

        self.showmap_button.on_click(on_button_clicked)
        display(out)
    


    

In [ ]:
merging_gui_jupiter()